In [41]:
import wandb
import pandas as pd

In [42]:

entity = "evangeorgerex"
project = "fwal"

sweep_ids = [
    # # FWAL models
    "ho0m3j5w", # FWAL
    # # "iw8hr7oj", # FWAl hierarchical
    # # "ryhck7mh", # FWAl hierarchical sigmoid 
    # # "bwhcxdf3", # FWAL hierarchical shared
    # # "54j0jkpy", # FWAL hierarchical shared sigmoid loss
    # # "9npryg91", # FWAL hierarchical TTI loss
    "x9s9n05b", # FWAL hierarchical TTI loss L1 ablation
    "7i2o46ri", # FWAL hierarchical TTI loss L1 ablation (updated TTI loss, still running)
    
    # Baselines
    "zkvbu18r", # lasso baseline 
    "22eckaxx", # rf baseline
    "gty91unf", # rf baseline
    "efjov5qc", # lasso baseline
    "zkvbu18r", # lasso baseline more
    "u0xyvsw9", # xgboost
    "8yghctpp", # supervised cae
    "0lh1b7hs", # SEFS
    "40yayf8d", # MLP
    "h245pc2i", # cae
    # finance:
    "18c8ij60", # xgboost finance
    "xclyvl0c", # supervised cae finance
    "2xn215vh", # SEFS finance
    "4upk4xrk", # RF finance
    "2bqm9fb6", # MLP finance
    "xpxe64ee", # LASSO finance
    
    # "91aly80j", # FWAL finance
    
    "jqulrkke", # cae finance
    # PBMC
    "vle39arl", # supervised cae
    "iq7hs7sp", # SEFS
    "tz0cpiih", # MLP
    "a600kww9", # cae
    "4yfu2hrj", # XGBoost PBMC
    # feature selection baselines
    "l102r3et", # rf
    "rdvu3fg8", # LASSO
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
]
'''
These sweeps are all broken because the reported bestmodel scores are wrong:
    # "ycv2gbna", # fwal only reconstruct masked mice protein more patience
    # "9tjs7cai", # fwal as mlp baseline
    # "nmsvf0u4", # fwal new datasets
    # "5kxy8gl8", # fwal as mlp baseline
    # "t1z9hleb", # fwal tti 
    # "3ubxo03v", # finish fwal tti
    # "t8i7yh69", # fwal gisette
    # "cwhd81fy", # fwal MNIST (states are failed despite being finished for some reason)
    # "z9b6sox8", # fwal only reconstruct masked
    # "xi4jr64q", # fwals only reconstruct masked GPU
    # "aie2q1ph", # cae
    # "uunpksug", # cae
'''


'\nThese sweeps are all broken because the reported bestmodel scores are wrong:\n    # "ycv2gbna", # fwal only reconstruct masked mice protein more patience\n    # "9tjs7cai", # fwal as mlp baseline\n    # "nmsvf0u4", # fwal new datasets\n    # "5kxy8gl8", # fwal as mlp baseline\n    # "t1z9hleb", # fwal tti \n    # "3ubxo03v", # finish fwal tti\n    # "t8i7yh69", # fwal gisette\n    # "cwhd81fy", # fwal MNIST (states are failed despite being finished for some reason)\n    # "z9b6sox8", # fwal only reconstruct masked\n    # "xi4jr64q", # fwals only reconstruct masked GPU\n    # "aie2q1ph", # cae\n    # "uunpksug", # cae\n'

In [43]:
def fetch_runs(entity, project, sweep_ids):
    api = wandb.Api()
    runs_data = []

    for sweep_id in sweep_ids:
        sweep_runs = api.sweep(f"{entity}/{project}/{sweep_id}").runs
        for run in sweep_runs:
            # Check if the run's state is 'finished'
            if run.state == "finished" or sweep_id == "cwhd81fy":
                dataset_name = run.config.get("dataset", "Unknown")
                if dataset_name in ["poly_binarised_decimalised_mod10_synth", "simple_trig_synth", "Unknown"]:
                    continue
                
                if run.config.get("as_MLP_baseline", False):
                    model_name = "MLP_same_capacity"
                else:
                    model_name = run.config.get("model", "Unknown")
                    if model_name == "fwal":
                        model_name = "F-Act"
                        
                    if run.config.get("hierarchical", False):
                        model_name = "F-Act hierarchical"
                        if run.config.get("share_mask", False):
                            model_name = "F-Act hierarchical shared"
                            if run.config.get("sigmoid_loss", False):
                                model_name = "F-Act hierarchical shared L_sig"
                    if sweep_id == "x9s9n05b":
                        model_name = "F-Act hierarchical TTI (old)"
                    if sweep_id in ["9npryg91", "7i2o46ri"]:
                        model_name = "F-Act hierarchical TTI"
                run_data = {
                    "model": model_name,  # Assuming model name is stored in 'model_name'
                    "dataset": dataset_name,  # Assuming dataset name is stored in 'dataset_name'
                    "seed": run.config.get("seed_model_init", None),
                    "bestmodel_train/F1_weighted": run.summary.get("bestmodel_train/F1_weighted", float('nan')),
                    "bestmodel_valid/F1_weighted": run.summary.get("bestmodel_valid/F1_weighted", float('nan')),
                    "bestmodel_test/F1_weighted": run.summary.get("bestmodel_test/F1_weighted", float('nan')),
                }
                
                
                # Specific hyperparameters
                if model_name == "lasso":
                    run_data["lasso_C"] = run.config.get("lasso_C")
                    run_data["lasso_l1_ratio"] = run.config.get("lasso_l1_ratio")
                elif model_name == "rf":
                    run_data["rf_max_depth"] = run.config.get("rf_max_depth")
                elif model_name in ["cae", "supervised_cae"]:
                    run_data["CAE_neurons_ratio"] = run.config.get("CAE_neurons_ratio")
                elif model_name == "F-Act hierarchical TTI":
                    run_data["sparsity_regularizer_hyperparam"] = run.config.get("sparsity_regularizer_hyperparam")
                elif model_name == "F-Act hierarchical TTI (old)":
                    run_data["sparsity_regularizer_hyperparam"] = run.config.get("sparsity_regularizer_hyperparam")
                    
                    
                runs_data.append(run_data)

    return runs_data

def process_runs_data(runs_data):
    df = pd.DataFrame(runs_data)
    
    # Handling models with hyperparameter sweeps separately
    models_with_hyperparams = ['lasso', 'rf', 'cae', 'supervised_cae', 'F-Act hierarchical TTI', 'F-Act hierarchical TTI (old)']
    best_hyperparams = {}
    
    for model in models_with_hyperparams:
        model_df = df[df['model'] == model]        
        
        # Further aggregate to find the best hyperparameter combination per dataset
        if model == 'lasso':
            best_combinations = model_df.groupby(['dataset', 'lasso_C', 'lasso_l1_ratio'])['bestmodel_valid/F1_weighted'].mean().reset_index()
        elif model == 'rf':
            best_combinations = model_df.groupby(['dataset', 'rf_max_depth'])['bestmodel_valid/F1_weighted'].mean().reset_index()
        elif model in ['cae', 'supervised_cae']:
            best_combinations = model_df.groupby(['dataset', 'CAE_neurons_ratio'])['bestmodel_valid/F1_weighted'].mean().reset_index()
        elif model == 'F-Act hierarchical TTI':
            best_combinations = model_df.groupby(['dataset', 'sparsity_regularizer_hyperparam'])['bestmodel_valid/F1_weighted'].mean().reset_index()
        elif model == 'F-Act hierarchical TTI (old)': # Same as above, but for the old TTI loss
            best_combinations = model_df.groupby(['dataset', 'sparsity_regularizer_hyperparam'])['bestmodel_valid/F1_weighted'].mean().reset_index()
        
        # Identify the best hyperparameters for each dataset
        best_combinations = best_combinations.loc[best_combinations.groupby('dataset')['bestmodel_valid/F1_weighted'].idxmax()]
        
        best_hyperparams[model] = best_combinations
        
        # Filter the original df to include only runs with the best hyperparameters
        if model == 'lasso':
            df = df[~((df['model'] == 'lasso') & ~df[['dataset', 'lasso_C', 'lasso_l1_ratio']].apply(tuple, 1).isin(best_combinations[['dataset', 'lasso_C', 'lasso_l1_ratio']].apply(tuple, 1)))]
        elif model == 'rf':
            df = df[~((df['model'] == 'rf') & ~df[['dataset', 'rf_max_depth']].apply(tuple, 1).isin(best_combinations[['dataset', 'rf_max_depth']].apply(tuple, 1)))]
        elif model == 'cae':
            df = df[~((df['model'] == 'cae') & ~df[['dataset', 'CAE_neurons_ratio']].apply(tuple, 1).isin(best_combinations[['dataset', 'CAE_neurons_ratio']].apply(tuple, 1)))]
        elif model == 'supervised_cae':
            df = df[~((df['model'] == 'supervised_cae') & ~df[['dataset', 'CAE_neurons_ratio']].apply(tuple, 1).isin(best_combinations[['dataset', 'CAE_neurons_ratio']].apply(tuple, 1)))]
        elif model == 'F-Act hierarchical TTI':
            df = df[~((df['model'] == 'F-Act hierarchical TTI') & ~df[['dataset', 'sparsity_regularizer_hyperparam']].apply(tuple, 1).isin(best_combinations[['dataset', 'sparsity_regularizer_hyperparam']].apply(tuple, 1)))]
        elif model == 'F-Act hierarchical TTI (old)':
            df = df[~((df['model'] == 'F-Act hierarchical TTI (old)') & ~df[['dataset', 'sparsity_regularizer_hyperparam']].apply(tuple, 1).isin(best_combinations[['dataset', 'sparsity_regularizer_hyperparam']].apply(tuple, 1)))]

    # Now, group by model and dataset to average across seeds for the final analysis
    final_grouped = df.groupby(['model', 'dataset']).mean().reset_index()
    
    # Printing best hyperparameters
    for model, params_df in best_hyperparams.items():
        print(f"Best hyperparameters for {model}:")
        print(params_df)
        print("\n")
    
    return final_grouped, best_hyperparams


# Fetch runs data
runs_data = fetch_runs(entity, project, sweep_ids)
# Process and group by model and dataset, then average
averaged_data, best_hyperparams = process_runs_data(runs_data)

wandb: WARNING A graphql request initiated by the public wandb API timed out (timeout=19 sec). Create a new API with an integer timeout larger than 19, e.g., `api = wandb.Api(timeout=29)` to increase the graphql timeout.


Best hyperparameters for lasso:
                              dataset  lasso_C  lasso_l1_ratio  \
2                              COIL20     10.0            0.50   
21                             Isolet    100.0            0.25   
30                              MNIST     10.0            0.00   
36                               PBMC     10.0            0.00   
54                               USPS     10.0            0.75   
68                            finance     10.0            0.50   
84                            gisette     10.0            0.75   
96                            madelon     10.0            0.00   
116                      mice_protein    100.0            0.00   
133  poly_binarised_decimalised_synth    100.0            0.50   

     bestmodel_valid/F1_weighted  
2                       0.982551  
21                      0.947942  
30                      0.911984  
36                      0.873614  
54                      0.934424  
68                      0.59504

In [44]:
best_hyperparams.keys()
lasso_hyperparams = best_hyperparams['lasso']
rf_hyperparams = best_hyperparams['rf']
cae_hyperparams = best_hyperparams['cae']
supervised_cae_hyperparams = best_hyperparams['supervised_cae']
Fact_hyperparams = best_hyperparams['F-Act hierarchical TTI']

In [45]:
lasso_hyperparams.style.set_table_styles(
    [{'selector': 'th', 'props': [('background', '#606060'), ('color', 'white')]}]
).set_caption("Lasso Best Hyperparameters")

,dataset,lasso_C,lasso_l1_ratio,bestmodel_valid/F1_weighted
2,COIL20,10.000000,0.500000,0.982551
21,Isolet,100.000000,0.250000,0.947942
30,MNIST,10.000000,0.000000,0.911984
36,PBMC,10.000000,0.000000,0.873614
54,USPS,10.000000,0.750000,0.934424
68,finance,10.000000,0.500000,0.595040
84,gisette,10.000000,0.750000,0.968095
96,madelon,10.000000,0.000000,0.548075
116,mice_protein,100.000000,0.000000,0.947724
133,poly_binarised_decimalised_synth,100.000000,0.500000,0.502693


In [46]:
rf_hyperparams.style.set_table_styles(
    [{'selector': 'th', 'props': [('background', '#606060'), ('color', 'white')]}]
).set_caption("RF Best Hyperparameters")

,dataset,rf_max_depth,bestmodel_valid/F1_weighted
2,COIL20,7.000000,0.970085
5,Isolet,7.000000,0.921524
8,MNIST,7.000000,0.910721
11,PBMC,7.000000,0.869854
14,USPS,7.000000,0.940427
17,finance,7.000000,0.600702
20,gisette,7.000000,0.949999
23,madelon,7.000000,0.693069
26,mice_protein,7.000000,0.962906
29,poly_binarised_decimalised_synth,7.000000,0.506308


In [47]:
cae_hyperparams.style.set_table_styles(
    [{'selector': 'th', 'props': [('background', '#606060'), ('color', 'white')]}]
).set_caption("CAE Best Hyperparameters")

,dataset,CAE_neurons_ratio,bestmodel_valid/F1_weighted
9,COIL20,1.000000,0.983572
17,Isolet,0.800000,0.828027
29,USPS,1.000000,0.904831
36,finance,0.700000,0.611487
49,madelon,1.000000,0.727068
55,mice_protein,0.600000,0.894743
67,poly_binarised_decimalised_synth,0.800000,0.419617


In [48]:
supervised_cae_hyperparams.style.set_table_styles(
    [{'selector': 'th', 'props': [('background', '#606060'), ('color', 'white')]}]
).set_caption("CAE Best Hyperparameters")

,dataset,CAE_neurons_ratio,bestmodel_valid/F1_weighted
0,COIL20,0.100000,0.058931
11,Isolet,0.200000,0.039760
20,PBMC,0.100000,0.832341
21,USPS,0.100000,0.212480
36,finance,0.600000,0.547139
50,madelon,1.000000,0.624305
60,mice_protein,1.000000,0.167532
63,poly_binarised_decimalised_synth,0.300000,0.368871


In [49]:
Fact_hyperparams.style.set_table_styles(
    [{'selector': 'th', 'props': [('background', '#606060'), ('color', 'white')]}]
).set_caption("F-Act hierarchical TTI")

,dataset,sparsity_regularizer_hyperparam,bestmodel_valid/F1_weighted
1,COIL20,0.100000,0.988301
8,Isolet,0.100000,0.888984
16,USPS,0.500000,0.959775
22,finance,0.100000,0.538629
29,madelon,0.100000,0.728721
38,mice_protein,1.000000,0.933865
44,poly_binarised_decimalised_synth,0.500000,0.157353


In [50]:


# Pivoting for 'bestmodel_train/F1_weighted'
train_pivot = averaged_data.pivot(index='model', columns='dataset', values='bestmodel_train/F1_weighted')
# print("Training F1 Weighted Averages:")
# print(train_pivot)
# print("\n")  # Add some space between tables

# Pivoting for 'bestmodel_valid/F1_weighted'
valid_pivot = averaged_data.pivot(index='model', columns='dataset', values='bestmodel_valid/F1_weighted')
# print("Validation F1 Weighted Averages:")
# print(valid_pivot)
# print("\n")  # Add some space between tables

# Pivoting for 'bestmodel_test/F1_weighted'
test_pivot = averaged_data.pivot(index='model', columns='dataset', values='bestmodel_test/F1_weighted')
# print("Test F1 Weighted Averages:")
# print(test_pivot)
for df in [train_pivot, valid_pivot, test_pivot]:
    df.drop('MNIST', axis=1, inplace=True)
    df.drop('gisette', axis=1, inplace=True) 
    df.drop('poly_binarised_decimalised_synth', axis=1, inplace=True)

def highlight_max(s):
    '''
    Highlight the maximum in a Series by changing font color to red and making it bold.
    '''
    is_max = s == s.max()
    return ['color: green; font-weight: bold' if v else '' for v in is_max]


table_styles = [
    {'selector': 'th',
     'props': [('background', '#606060'), ('color', 'white')]
    }
]

In [51]:
train_pivot.style.set_table_styles(table_styles).set_caption("Training F1 Weighted Averages").apply(highlight_max)

dataset,COIL20,Isolet,PBMC,USPS,finance,madelon,mice_protein
model,,,,,,,
F-Act,1.000000,1.000000,nan,0.997641,nan,1.000000,0.998956
F-Act hierarchical TTI,1.000000,0.989935,nan,0.990563,0.581817,0.869327,0.981255
F-Act hierarchical TTI (old),1.000000,0.999628,nan,0.999091,0.628549,0.935525,0.993242
MLP_same_capacity,0.998274,0.998511,0.978579,0.982363,0.339587,0.820071,1.000000
SEFS,0.964073,0.935815,nan,0.948776,nan,0.726851,0.883796
cae,1.000000,0.962531,nan,0.930945,0.656600,0.873945,0.962683
lasso,1.000000,1.000000,1.000000,0.994983,0.670358,0.803830,0.985080
rf,0.986327,0.984742,0.937396,0.957755,0.877761,0.995769,0.999807
supervised_cae,0.054281,0.040158,0.830378,0.207287,0.537526,0.610881,0.191261


In [52]:
valid_df = valid_pivot.style.set_table_styles(table_styles).set_caption("Valid F1 Weighted Averages").apply(highlight_max)
valid_df

dataset,COIL20,Isolet,PBMC,USPS,finance,madelon,mice_protein
model,,,,,,,
F-Act,0.994076,0.921656,nan,0.968271,nan,0.598737,0.989262
F-Act hierarchical TTI,0.988301,0.888984,nan,0.959775,0.538629,0.728721,0.933865
F-Act hierarchical TTI (old),0.990732,0.893329,nan,0.971657,0.579261,0.776570,0.975395
MLP_same_capacity,0.987963,0.906103,0.882365,0.966356,0.341450,0.568512,0.975422
SEFS,0.945834,0.817430,nan,0.940525,nan,0.650343,0.858909
cae,0.983572,0.828027,nan,0.904831,0.611487,0.727068,0.894743
lasso,0.982551,0.947942,0.873614,0.934424,0.595040,0.548075,0.947724
rf,0.970085,0.921524,0.869854,0.940427,0.600702,0.693069,0.962906
supervised_cae,0.058931,0.039760,0.832341,0.212480,0.547139,0.624305,0.167532


In [53]:
test_df = test_pivot.style.set_table_styles(table_styles).set_caption("Test F1 Weighted Averages").apply(highlight_max)
test_df

dataset,COIL20,Isolet,PBMC,USPS,finance,madelon,mice_protein
model,,,,,,,
F-Act,0.991837,0.919414,nan,0.964203,nan,0.578604,0.978521
F-Act hierarchical TTI,0.983769,0.860279,nan,0.958703,0.538603,0.722480,0.916380
F-Act hierarchical TTI (old),0.986036,0.878258,nan,0.966338,0.587349,0.770906,0.964669
MLP_same_capacity,0.983620,0.895551,0.887427,0.959278,0.336238,0.565471,0.972048
SEFS,0.943247,0.801379,nan,0.933565,nan,0.613505,0.820170
cae,0.970427,0.801414,nan,0.904749,0.592564,0.701883,0.853472
lasso,0.982413,0.945832,0.892459,0.933578,0.597777,0.515320,0.944755
rf,0.967561,0.900926,0.886592,0.933634,0.619456,0.671901,0.969803
supervised_cae,0.064637,0.036833,0.853663,0.209018,0.544434,0.618427,0.172394


In [54]:
print(test_df.to_latex())

\begin{table}
\caption{Test F1 Weighted Averages}
\th#606060
\begin{tabular}{lrrrrrrr}
dataset & COIL20 & Isolet & PBMC & USPS & finance & madelon & mice_protein \\
model &  &  &  &  &  &  &  \\
F-Act & \colorgreen \font-weightbold 0.991837 & 0.919414 & nan & 0.964203 & nan & 0.578604 & \colorgreen \font-weightbold 0.978521 \\
F-Act hierarchical TTI & 0.983769 & 0.860279 & nan & 0.958703 & 0.538603 & 0.722480 & 0.916380 \\
F-Act hierarchical TTI (old) & 0.986036 & 0.878258 & nan & 0.966338 & 0.587349 & 0.770906 & 0.964669 \\
MLP_same_capacity & 0.983620 & 0.895551 & 0.887427 & 0.959278 & 0.336238 & 0.565471 & 0.972048 \\
SEFS & 0.943247 & 0.801379 & nan & 0.933565 & nan & 0.613505 & 0.820170 \\
cae & 0.970427 & 0.801414 & nan & 0.904749 & 0.592564 & 0.701883 & 0.853472 \\
lasso & 0.982413 & \colorgreen \font-weightbold 0.945832 & 0.892459 & 0.933578 & 0.597777 & 0.515320 & 0.944755 \\
rf & 0.967561 & 0.900926 & 0.886592 & 0.933634 & \colorgreen \font-weightbold 0.619456 & 0.671901 & 0.